# Continuous Control

---




# DDPG Agent 

In [6]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import deque
from statistics import mean, stdev

from ENV import environment_loader

In [7]:
env, state_size, action_size, brain_name, num_agents = environment_loader("Reacher_Single/Reacher.exe", no_graphics = True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00

In [8]:
def DDPG_eval(agent, env, max_t=100):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    episode_length = 0                                     # measure number of steps allowed before done flag
    for i in range(max_t):
        # actions = np.random.randn(num_agents, action_size) # select an action (for each agent); randomly!
        actions = agent.act(states[0]) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        episode_length +=1
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Episode length is {}'.format(episode_length))
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [9]:
score_history = []

def DDPG_train(agent, env, max_t=100, num_episodes = 1, print_every=100):
    score_history = []
    scores_deque = deque(maxlen=print_every)
    last_running_mean = float('-inf')

    for episode in range(num_episodes):
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        

        for i in range(max_t):
            actions = agent.act(states[0]) # select an action (for each agent)
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones)

            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        

        returns_in_episode = np.mean(scores)
        scores_deque.append(returns_in_episode)
        score_history.append(returns_in_episode)
        if episode > print_every:
            if mean(scores_deque) > last_running_mean:
                    print('Last {} was better, going to save it'.format(print_every))
                    torch.save(new_agent.actor_local.state_dict(), 'checkpoint_actor.pth')
                    torch.save(new_agent.critic_local.state_dict(), 'checkpoint_critic.pth')
                    last_running_mean = mean(scores_deque)

        print("\r", 'Total score (averaged over agents) {} episode: {}'.format(episode, returns_in_episode), end="")
    

    return score_history

In [10]:
import os.path

filename="checkpoint_actor.pth"
path = ""

def fileAtLocation(filename, path):
    return os.path.exists(path + filename)

def load_previous(new_agent):
    loaded_agent = new_agent
    if fileAtLocation(filename, path):
        print("Found previous trained Agent, going to load them!")
        loaded_agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
        loaded_agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))
    return loaded_agent

In [11]:
%load_ext autoreload
%autoreload 2
from AGENT import DDPG_Agent

new_agent = DDPG_Agent(state_size, action_size, 5)
##new_agent = load_previous(new_agent)


--- Agent Params ---
Going to train on cpu
Learning Rate:: Actor: 0.0001 | Critic: 0.001
Replay Buffer:: Buffer Size: 100000 | Sampled Batch size: 128


In [7]:
##env.no_graphics = True
score_history = DDPG_train(new_agent, env, max_t = 300, num_episodes = 150)
print(score_history)

C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
 Total score (averaged over agents) 13 episode: 0.0

KeyboardInterrupt: 

In [ ]:
print(overall_score)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(overall_score)+1), overall_score)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [6]:
%load_ext autoreload
%autoreload 2
from AGENT import DDPG_Agent

new_agent = DDPG_Agent(state_size, action_size, 5)
new_agent = load_previous(new_agent)


--- Agent Params ---
Going to train on cpu
Learning Rate:: Actor: 0.0001 | Critic: 0.001
Replay Buffer:: Buffer Size: 100000 | Sampled Batch size: 128
Found previous trained Agent, going to load them!


In [14]:
DDPG_eval(new_agent, env, max_t = 100000000000)

C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Episode length is 1001
Total score (averaged over agents) this episode: 0.5299999881535769


In [ ]:
env.close()

In [12]:
torch.cuda.is_available()

False